In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

In [2]:
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Input
from keras.optimizers import Adam
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K

Using TensorFlow backend.


In [3]:
from google.colab import drive
drive.mount('/content/drive/')
DATA_PATH = '/content/drive/My Drive/ReCognizer_for_Cabinet/datasets/'
!ls '/content/drive/My Drive/ReCognizer_for_Cabinet'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/
datasets


In [0]:
NUM_MEMBERS = 4
IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224

NUM_TRAINS = 1600
NUM_VALS = 400
BATCH_SIZE = 128

In [5]:
# model architecture
input_tensor = Input(shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 3))
base_model = VGG16(weights='imagenet',
                   include_top=False,
                   input_tensor=input_tensor)

# freezing default vgg16 layers
for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
pred = Dense(NUM_MEMBERS, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=pred)
model.compile(Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()





58892288/58889256 [==============================] - 5s 0us/step








Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
________________________________________________

In [10]:
# train and test data generator
train_datagen = ImageDataGenerator(
                    rescale=1.0 / 255,
                    shear_range=0.2,
                    zoom_range=0.2,
                    horizontal_flip=True,
                    rotation_range=30
                )

test_datagen = ImageDataGenerator(
                    rescale=1.0 / 255
                )

# pick datasets from directory
train_generator = train_datagen.flow_from_directory(
                    DATA_PATH + 'train',
                    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
                    batch_size=BATCH_SIZE,
                    class_mode='categorical',
                    shuffle=True
                )

val_generator = test_datagen.flow_from_directory(
                    DATA_PATH + 'val',
                    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
                    batch_size=BATCH_SIZE,
                    class_mode='categorical',
                    shuffle=True
                )

Found 238 images belonging to 4 classes.
Found 150 images belonging to 4 classes.


In [12]:
history = model.fit_generator(train_generator,
                              steps_per_epoch=NUM_TRAINS // BATCH_SIZE,
                              epochs=3,
                              verbose=1,
                              validation_data=val_generator,
                              validation_steps=NUM_VALS // BATCH_SIZE,
                             )

Epoch 1/3
12/12 [==============================] - 20s 2s/step - loss: 0.1865 - acc: 0.9973 - val_loss: 0.1687 - val_acc: 1.0000
Epoch 2/3
12/12 [==============================] - 19s 2s/step - loss: 0.1264 - acc: 0.9972 - val_loss: 0.1124 - val_acc: 1.0000
Epoch 3/3
12/12 [==============================] - 20s 2s/step - loss: 0.0923 - acc: 0.9992 - val_loss: 0.0861 - val_acc: 1.0000


In [0]:
model.save('rcc_members.h5')